In [1]:
from avrpy import *

## Communication logging

You can log all serial port communication by setting the `log_file` argument when you connect to the AVR device.

In [2]:
log_file = None                   # no logging
log_file = 'sync_device log.txt'  # save to file
log_file = 'print'                # print here

In [3]:
if 'avr' in globals():
    if not avr.com.closed:
        avr.com.close()
avr = Mega2560("COM3", log_file=log_file)

14:06:53 RX: Sync device is ready. Firmware version: 0.4.2



# Setup laser shutters
`set_shutters()` sends command `L`, followed by 1 byte specifying used lasers (`0 0 0 0 Cy2 Cy3 Cy5 Cy7`), and a bool indicating whether ALEX is used.

After that, you can open laser shutters using function `open_shutters()` (command `M`, no arguments), and close using `stop()` (command `Q`, no arguments).

In [4]:
# only Cy7 channel
avr.set_shutters([0, 0, 0, 1])

14:06:53 TX: b'L\x08\x00\x00\x00' -> L 8
14:06:53 RX: OK



In [5]:
avr.open_shutters()

14:06:53 TX: b'M\x00\x00\x00\x00' -> M 0
14:06:53 RX: OK



In [6]:
# close shutters, go back to the IDLE state
avr.stop()

14:06:53 TX: b'Q\x00\x00\x00\x00' -> Q 0
14:06:53 RX: OK



In [7]:
# Cy2, Cy5, Cy7, and ALEX
avr.set_shutters([1, 0, 1, 1], True)
avr.open_shutters()

14:06:53 TX: b'L\r\x01\x00\x00'   -> L 269
14:06:53 RX: OK

14:06:53 TX: b'M\x00\x00\x00\x00' -> M 0
14:06:53 RX: OK



In [8]:
# This fails because shutters are already open
avr.open_shutters()

14:06:53 TX: b'M\x00\x00\x00\x00' -> M 0
14:06:53 RX: M: Not in the IDLE state



SyncDeviceError: Incorrect args supplied to the sync device.
Device reply:
 -> M: Not in the IDLE state

# Continuous acquisition

For continuous acquisition, we need to set
* Laser shutters, `set_shutters()`, command `L`, with at least one laser and  `ALEX=False`.
* Laser exposure time, `exp_time_us`, command `E`.
* Camera read-out interval, `cam_readout_us`, command `I` (important only if using ROI and imaging faster than 12ms). Note that it also defines the duration of the first discard frame.
* Number of frames - argument of `start_continuous`, command `C`, which starts continuous acquisition.

In [9]:
avr.stop()
# Cy2, Cy3, Cy7
avr.set_shutters([1, 1, 0, 1])
avr.exp_time_us = 125_000       # 125 ms
avr.cam_readout_us = 2_500      # 2.5 ms camera read-out time

14:07:03 TX: b'Q\x00\x00\x00\x00' -> Q 0
14:07:03 RX: OK

14:07:03 TX: b'L\x0b\x00\x00\x00' -> L 11
14:07:03 RX: OK

14:07:03 TX: b'EH\xe8\x01\x00'    -> E 125000
14:07:03 RX: OK

14:07:03 TX: b'I\xc4\t\x00\x00'   -> I 2500
14:07:03 RX: OK



In [10]:
# Acquire 20 frames
avr.start_continuous(40)

14:07:04 TX: b'C(\x00\x00\x00'    -> C 40
14:07:04 RX: OK



# Setup stroboscopic time-lapse acquisition

For time-lapse, there are two separate time periods: the laser exposure time, defined as the duration of the open shutter, and the frame period - time period between two subsequent frames. The latter one is set using `acq_period_us`, command `A`. After that, start imaging using `start_stroboscopic()`, command `S`, and provide number of frames as argument.

In [11]:
# Laser exposure of 400ms, time-lapse every 2 s
avr.acq_period_us = 2000_000
avr.exp_time_us = 400_000
# Acquire 6 frames
avr.start_stroboscopic(6)

14:07:05 TX: b'A\x80\x84\x1e\x00' -> A 2000000
14:07:05 RX: OK

14:07:05 TX: b'E\x80\x1a\x06\x00' -> E 400000
14:07:05 RX: OK

14:07:05 TX: b'S\x06\x00\x00\x00' -> S 6
14:07:05 RX: OK



# Setup stroboscopic ALEX acquisition
This is done the same way as time-lapse, except that you need to activate ALEX when setting laser shutters.

In [12]:
# Acquire 6 frames with alternating Cy3/Cy5/Cy7
avr.set_shutters([0, 1, 1, 1], ALEX=True)
avr.start_stroboscopic(6)

14:07:06 TX: b'L\x0e\x01\x00\x00' -> L 270
14:07:06 RX: OK

14:07:06 TX: b'S\x06\x00\x00\x00' -> S 6
14:07:06 RX: OK



Note that you MUST ensure that the cycle duration is long enough to alternate between the lasers, otherwise the device might skip frames. Make sure that for $N$ spectral channels, the cycle duration is at least $N*(e + r + s)$, where $e$ is the camera exposure time, $r$ is the camera read-out time, and $s$ is the shutter delay.

In [13]:
# Acquire 5 frames with alternating Cy2/Cy3/Cy5/Cy7, 200 ms exposure, as fast as possible

# We have a SLOW CAMERA with 200 ms read-out time
shutters = [1, 1, 1, 1]
N = shutters.count(1)
avr.set_shutters(shutters, ALEX=True)

e, r, s = 400_000, 200_000, 2_000  # exposure, read-out, and shutter delay times

avr.exp_time_us = e
avr.cam_readout_us = r
avr.shutter_delay_us = s
avr.acq_period_us = N*(e + r + s)

avr.start_stroboscopic(5)

14:07:07 TX: b'L\x0f\x01\x00\x00' -> L 271
14:07:07 RX: OK

14:07:07 TX: b'E\x80\x1a\x06\x00' -> E 400000
14:07:07 RX: OK

14:07:07 TX: b'I@\r\x03\x00'      -> I 200000
14:07:07 RX: OK

14:07:07 TX: b'D\xd0\x07\x00\x00' -> D 2000
14:07:07 RX: OK

14:07:07 TX: b'A@\xbe$\x00'       -> A 2408000
14:07:07 RX: OK

14:07:07 TX: b'S\x05\x00\x00\x00' -> S 5
14:07:07 RX: OK



In [14]:
# Example with skipped frames where the acqusition period is not long enough
avr.acq_period_us = N*avr.exp_time_us
avr.start_stroboscopic(3)

14:07:08 TX: b'A\x00j\x18\x00'    -> A 1600000
14:07:08 RX: OK

14:07:08 TX: b'S\x03\x00\x00\x00' -> S 3
14:07:08 RX: OK



# ALEX and time-lapse
By setting `acq_period_us` to be extra long, we combine ALEX and time-lapse imaging.

In [15]:
avr.acq_period_us = N*(e + r + s) + 2_000_000
print(f"{N} channels, {avr.exp_time_us/1000} ms exposure time per channel, every {avr.acq_period_us/1e6} s")
avr.start_stroboscopic(5)

14:07:10 TX: b'A\xc0BC\x00'       -> A 4408000
14:07:10 RX: OK

4 channels, 400.0 ms exposure time per channel, every 4.408 s
14:07:10 TX: b'S\x05\x00\x00\x00' -> S 5
14:07:10 RX: OK



In [16]:
# Fast example
# Acquire 100 frames with fast alternating Cy2/Cy3/Cy5/Cy7, 4 ms exposure, burst every 50 ms

avr.set_shutters([1, 1, 1, 1], ALEX=True)

avr.exp_time_us = 4_000
avr.cam_readout_us = 2_000
avr.shutter_delay_us = 1_000
avr.acq_period_us = 50_000

avr.start_stroboscopic(100)

14:07:10 TX: b'L\x0f\x01\x00\x00' -> L 271
14:07:10 RX: OK

14:07:10 TX: b'E\xa0\x0f\x00\x00' -> E 4000
14:07:10 RX: OK

14:07:10 TX: b'I\xd0\x07\x00\x00' -> I 2000
14:07:10 RX: OK

14:07:10 TX: b'D\xe8\x03\x00\x00' -> D 1000
14:07:10 RX: OK

14:07:10 TX: b'AP\xc3\x00\x00'    -> A 50000
14:07:10 RX: OK

14:07:10 TX: b'Sd\x00\x00\x00'    -> S 100
14:07:10 RX: OK



# Device status
Currently, we don't have any mechanism to ask the device about the status of the acquisition. However, it sends "DONE" back when the acquisition is completed.

In [17]:
avr.com.read_all()

b'DONE\n'